# File-io Solutions

In [ ]:
from astropy.io import fits
import numpy as np
import pylab as plt
from scipy.signal import medfilt
import numpy as np
from scipy.interpolate import interp1d

# Warm Up

# #1

    a) 
       1. Images of galaxies, stars, quasars
       2. BinaryTables: usually holding spectrum information
       3. PrimaryHDU: Information of telescope/viewing, (RA, DEC), seeing, time of observation, 
          who did it, filename 

    b) Two ways: data = hdu[number].data
                 data = fits.getdata(filename)

    c) The information is listed in the PrimaryHDU and can be some useful stuff in hdu.info() as 
       this tells you extensions of the fits files and what they contain. To access information you can use
       column names or  

    d) Most have multiple and can be checked using the hdu.info() command or looking at 
       header and seeing a T on extensions


# #2

In [ ]:
#below are a list of code with an empty comment above the line. Your task is to describe what each line is doing
#please be as specific as you can. Write the description in the comment. 


#This opens up the hdu object and assigns it to variable x
x = fits.open('Spectrum.fits')
    
#This command tells you the information contained within the fits file and the number of extensions
print(x.info())
    
#this assigns to a the header of the PrimaryHDU
a = x[0].header

#This lists out the header so that we can see it
print(repr(a))

#this assigns ot b the header of the first extension 
b = x[1].header
       
#this list the column names for the header of the first extension
print(list(b.keys()))
    
#This collects from the file the SPEC data
data = x[1].data['SPEC']
    
#This collects the WAVLN data from the file
data1 = x[1].data['WAVELN']

#This command closes the file
x.close()

# #3

In [ ]:
hdu = fits.open('dss.14.4.4.98432000000392+0.59.53.3939999999998.fits')

#How many things are in the FITS file? Please print out the names below
'''
2 things are in the fits files: with names PRIMARY and Photometric CALTABLE


'''

#What are the dimensions of each thing?
'''
PRIMARY has dimension of 177 X 177

Photometric CALTABLE has dimensions of 15 rows and 4 columns
'''




#Please write code that extracts the data from PrimaryHDU
data = hdu[0].data




#Please write code that extracts the columns from the BinTableHDU, what are their names and units associated with them
col = hdu[1].data.columns

I must stress that the solutions presented below are only one way to approach these problems and are by no means the only way to get the same answer. If you have something different but get the same result thats is perfectly okay, so long as the main points are there.

# 1 Data from Fits (Soln)

In [ ]:
#creating a variable that will hold the filename
file = 'first_14dec17.fits'


def get_data(filename):
    '''
    This function will print out the RA and DEC of all objects that have a peak flux of 100 mJy or more
    
    Parameters:
    filename: the name of the file we want to apply this function to
    
    Output:
    On the screen a list of RA and DEC 
    '''
    #makes an hdu object for the datafile
    hdu = fits.open(filename)
    
    #getting the RA, DEC and peak flux from the data
    RA = hdu[1].data['RA']
    DEC = hdu[1].data['DEC']
    FPEAK = hdu[1].data['FPEAK']
    
    #making a boolean filter the length of my data array
    bool_filter = np.zeros(len(FPEAK), dtype = bool)
    
    #this for loop checks to see if the condition FPEAK[i] >= 100 holds 
    #and assigns boolean filter the value of True
    for i in range(len(FPEAK)):
        
        #checking condition here
        if FPEAK[i] >= 100:
            
            #assigning boolean filter to True because this is the index we want to keep
            bool_filter[i] = True
    
    #simplifying the data set to only those that match criteria
    data_RA = RA[bool_filter]
    data_DEC = DEC[bool_filter]
    
    #printing out the RA and DEC
    for i in range(len(data_RA)):
        print('RA: %.2f, DEC: % .2f'%(data_RA[i], data_DEC[i]) )   

In [ ]:
def flux_southern(filename):
    '''
    This function will print out the total flux recieved in the southern hemisphere
    
    Parameters:
    filename: the name of the file we want to apply this function to
    
    Output:
    the total flux recieved in 
    '''
    #makes an hdu object for the datafile
    hdu = fits.open(filename)
    
    #getting the DEC and flux intensity from the file hdu
    DEC = hdu[1].data['DEC']
    FINT = hdu[1].data['FINT']

    #making a boolean filter the length of my data array
    bool_filter = np.zeros(len(FINT), dtype = bool)
    
    for i in range(len(FINT)):
        if DEC[i] < 0:
            bool_filter[i] = True
            
    south_flux = FINT[bool_filter]
    
    total_south_flux = np.sum(south_flux)
    
    
    
    print('%.3f mJy' % (total_south_flux))
    
    

In [ ]:
get_data(file)

In [ ]:
flux_southern(file)

In [ ]:
hdu = fits.open('star1.fits')
hdr = hdu[1].header

#print(repr(hdr))

col = hdu[1].columns
print(col.names[0])

# 2 Working with Spectrum Data

In [ ]:
files = ['star1.fits', 'star2.fits', 'galaxy1.fits', 'galaxy2.fits']

def plotting_spectrums(files):
    
    '''
    Function that will take in all the files above and plots them along with some well looked for lines
    
    Parameter
    -----------
    files: this is a list of all the file names which we will use to ultimately plot them
    
    Output:
    A subplot with all of the files along with lines
    
    '''
    
    
    spec_data = []
    wavelength = []
    
    
    for i in files:
        hdu = fits.open(i)
        
        data = medfilt(hdu[1].data['flux'], kernel_size = 5)
        lam = hdu[1].data['loglam']
        
        hdu.close()
        
        spec_data.append(data)
        wavelength.append(lam)
        
    data_array = np.array(spec_data)
    wavln_array = 10 ** np.array(wavelength)
    
    lines = [3727.1, 3729.9, 4364.4, 4687, 4862.7, 4960.3, 5008.2, 5413, 6564.6, 7137.8]
    
    fig = plt.figure(figsize = (20, 15))
    
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212, sharex = ax1)
    
    ax1.set_title('Star 1 Spectrum', fontsize = 15)
    ax1.set_xlabel(r'$ Wavelength [\AA]$ ')
    ax1.set_ylabel(r'$ Intensity \hspace{.5} 10^{-17} \frac{ergs}{s\hspace{.5} cm^2/ \AA}$', fontsize = 20)
    ax1.plot(wavln_array[0], data_array[0])
    ax1.set_xlim(wavln_array[0][0],7500) 
    
    for i in lines:
        
        ax1.axvline(i)
        ax2.axvline(i)
    
    ax2.set_title('Star 2 Spectrum', fontsize = 15)
    ax2.set_xlabel(r'$ Wavelength [\AA]$ ')
    ax2.set_ylabel(r'$ Intensity [\hspace{.5} 10^{-17} \frac{ergs}{s\hspace{.5} cm^2/ \AA}$', fontsize = 20)
    ax2.plot(wavln_array[1], data_array[1])
    
    fig.tight_layout()
    plt.show()
    
    fig1 = plt.figure(figsize = (20, 15))
    ax11 = fig1.add_subplot(211)
    ax22 = fig1.add_subplot(212, sharex = ax11)
    
    ax11.set_title('Galaxy 1 Spectrum', fontsize = 15)
    ax11.set_xlabel(r'$ Wavelength [\AA]$ ')
    ax11.set_ylabel(r'$ Intensity \hspace{.5} 10^{-17} \frac{ergs}{s\hspace{.5} cm^2/ \AA}$', fontsize = 20)
    ax11.plot(wavln_array[2], data_array[2])
    ax11.set_xlim(wavln_array[2][0], 7500)
    for i in lines:
        
        ax11.axvline(i)
        ax22.axvline(i)
    
    ax22.set_title('Galaxy 2 Spectrum', fontsize = 15)
    ax22.set_xlabel(r'$ Wavelength [\AA]$ ')
    ax22.set_ylabel(r'$ Intensity \hspace{.5} 10^{-17} \frac{ergs}{s\hspace{.5} cm^2/ \AA}$', fontsize = 20)
    ax22.plot(wavln_array[3], data_array[3])
    
   
    
    fig1.tight_layout()
    
    
    plt.show()
    
    
    
                

In [ ]:
plotting_spectrums(files)

Doppler Velocity

In [ ]:
galaxy2 = 'galaxy2.fits'

def doppler(filename):
    c = 3e8
    rest_fram_freq = c/7137.8e-10 #converting to the right units
    hdu = fits.open(filename)
        
    data = hdu[1].data['flux']
    lam = 10 ** hdu[1].data['loglam']/(1e10)
        
    hdu.close()
    freq_arr = c/lam
    
    filt = np.zeros(len(freq_arr), dtype = bool)
    
    for i in range(len(freq_arr)):
        if freq_arr[i] >= rest_fram_freq-5e13 and freq_arr[i] <= rest_fram_freq+5e13:
            filt[i] = True
            
    reduced_data = data[filt]
    reduced_freq = freq_arr[filt]
    
    delta_nu = rest_fram_freq - reduced_freq
    vel = -c * (delta_nu)/rest_fram_freq
    
    max_index = np.argmax(reduced_data)
    
    print('Doppler velocity of the galaxy is about %7.2f km/s' %(vel[max_index]/1000))

In [ ]:
doppler(galaxy2)  

# 3 Data Reduction of a Spectrum 

In [ ]:
def avg_spectrum(file):
    
    hdu = fits.open(file)
    
    sum_of_spectra = 0
    Num_Spec = hdu[0].header['NSPEC']
    
    for i in range(1, Num_Spec+1):
        
        sum_of_spectra += hdu[i].data['auto0_real']
        
    avg_spec = sum_of_spectra/Num_Spec
    
    return avg_spec

on_data = avg_spectrum('noise_on.fits')
off_data = avg_spectrum('noise_off.fits')

hdu_on = fits.open('noise_on.fits')
#making the x-axis array for you to use in the plot:
bw = hdu_on[0].header['BW']/1e6
n = hdu_on[0].header['nchan']
#This is the frequency array and has units of MHz be sure to label this in your plot
xaxis = np.linspace(-bw/2, bw/2, n) + (2*635) + 150 

plt.figure(figsize = (14, 12))
plt.title('Comparing On and Off Spectrum (avg-ed)')
plt.ylabel('Power')
plt.plot(xaxis, on_data, label = 'On Data')
plt.plot(xaxis, off_data, label = 'Off Data')
plt.legend(loc = 'best')


# Gain

In [ ]:
def Gain(s_on, s_off):
    
    T_sys = 30
    T_cold = 3
    
    diff_T = T_sys - T_cold
    diff_spec = s_on - s_off
    
    sum_diff_spec = np.sum(diff_spec)
    
    G = (diff_T/sum_diff_spec) * np.sum(s_off)
    
    return G


In [ ]:
print('The Gain for this problem is: %4.2f' %(Gain(on_data, off_data)))

# Casseopia

In [ ]:
avg_cass = avg_spectrum('cassiopea.fits')

In [ ]:
smoothed_cass = medfilt(avg_cass, kernel_size = 5)

In [ ]:
bool_filter = np.ones(len(smoothed_cass), dtype = bool)

for i in range(len(smoothed_cass)):
    if xaxis[i] >= 1420 and xaxis[i] <= 1421.5:
        bool_filter[i] = False
        
interp_data = smoothed_cass[bool_filter]

In [ ]:
xnew = xaxis[bool_filter]
y = interp1d(xnew, interp_data, kind = 'cubic')

centered_data = smoothed_cass - y(xaxis)

plt.figure(figsize = (14, 8))
plt.title('Centered Spectrum')
plt.xlabel('Frequency')
plt.ylabel('Power')
plt.plot(xaxis, centered_data)

# Putting it all Together

In [ ]:
G = Gain(on_data, off_data)
final_spec = G* centered_data

plt.figure(figsize = (14, 8))
plt.title('The Final Spectrum ')
plt.xlabel('Frequency')
plt.ylabel('Power [K]')
plt.plot(xaxis, final_spec)